In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install pdfminer

     |████████████████████████████████| 4.2MB 7.6MB/s 
     |████████████████████████████████| 1.9MB 37.6MB/s 
  Created wheel for pdfminer: filename=pdfminer-20191125-cp37-none-any.whl size=6140088 sha256=9e02cfefd28a9afe5d4b9e43e9c97f61664a4f721c67488cf04c7e71616d5c38
  Stored in directory: /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326
Successfully built pdfminer


In [3]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
 
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
 
            text = fake_file_handle.getvalue()
            yield text
 
            # close open handles
            converter.close()
            fake_file_handle.close()
 
def extract_text(pdf_path):
    for page in extract_text_by_page(pdf_path):
        print(page)
        print()

def extract_pages(pdf_path):
  pages=[]
  for page in extract_text_by_page(pdf_path):
      pages.append(page)
  return pages

In [5]:
import pandas as pd
import numpy as np
import os
import sys
import re

Объявляем набор данных

In [6]:
df = pd.DataFrame(columns=['Name','Sex', 'Age','Children'])

Путь к папке с файлами PDF

In [7]:
rootFolderPath = '/content/gdrive/MyDrive/DataSet'

In [42]:
def readTextPage(text):
  inputString = text
  NameStr =''
  SexStr = ''
  AgeStr = ''
  ChildrenStr = ''
  nameResult = re.search('Name: ', inputString)
  if nameResult!=None:
    inputString = inputString[nameResult.end():]
    #print('1: ',inputString)

  sexResult = re.search(' Sex: ',inputString)
  if sexResult!=None:
    NameStr = inputString[:sexResult.start()]
    inputString = inputString[sexResult.end():]
    #print('Имя: ',NameStr)

  ageResult = re.search(' Age: ',inputString)
  if ageResult!=None:
    SexStr = inputString[:ageResult.start()]
    inputString = inputString[ageResult.end():]
    #print('Пол: ',SexStr)

  childResult = re.search(' Children: ',inputString)
  if childResult!=None:
    AgeStr = inputString[:childResult.start()]
    inputString = inputString[childResult.end():]
    #print('Возраст: ',AgeStr)
    AgeStr = int(AgeStr)

    ChildrenStr = inputString
    #print('Детей: ',ChildrenStr)
    ChildrenStr =int(ChildrenStr)

  return NameStr, SexStr, AgeStr, ChildrenStr

In [44]:
#'Name','Sex', 'Age','Children'
for item in os.listdir(rootFolderPath):
  filePath = os.path.join(rootFolderPath,item)
  #print(filePath)
  text = extract_pages(filePath)
  for page in text:
    print(page)
    nameStr, sexStr, ageStr, childrenStr = readTextPage(page)
    df = df.append({'Name': nameStr, 'Sex' : sexStr, 'Age' : ageStr, 
                    'Children' : childrenStr}, ignore_index = True)

Name: Maria Sex: Female Age: 47 Children: 2 
Имя:  Maria
Пол:  Female
Возраст:  47
Детей:  2 
Name: Anna Sex: Female Age: 22 Children: 1 
Имя:  Anna
Пол:  Female
Возраст:  22
Детей:  1 
Name: Marc Sex: Male Age: 30 Children: 1 
Имя:  Marc
Пол:  Male
Возраст:  30
Детей:  1 
Name: Olga Sex: Female Age: 18 Children: 0   
Имя:  Olga
Пол:  Female
Возраст:  18
Детей:  0   
Name: Alex Sex: Male Age: 67 Children: 3  
Имя:  Alex
Пол:  Male
Возраст:  67
Детей:  3  


In [45]:
df.head()

,Name,Sex,Age,Children
0,Maria,Female,47,2
1,Anna,Female,22,1
2,Marc,Male,30,1
3,Olga,Female,18,0
4,Alex,Male,67,3
